# Level 6: MCP Based RAG (Medium Difficulty)

This notebook is an extension of the [Level 5 Agentic & MCP notebook](./Level5_agents_and_mcp.ipynb) with the addition of RAG.
This tutorial is for developers who are already familiar with [agentic RAG workflows](./Level4_RAG_agent.ipynb). This tutorial will highlight a couple of slightly more advanced use cases for agents where a single tool call is insufficient to complete the required task. Here we will rely on both agentic RAG and MCP server to expand our agents capabilities.

## Overview

This tutorial covers the following steps:
1. Review OpenShift logs for a failing pod.
2. Categorize the pod and summarize its error.
3. Search available troubleshooting documentations for ideas on how to resolve the error.
4. Send a Slack message to the ops team with a brief summary of the error and next steps to take.

### MCP Tools:

#### OpenShift MCP Server
Throughout this notebook we will be relying on the [kuberenetes-mcp-server](https://github.com/manusa/kubernetes-mcp-server) by [manusa](https://github.com/manusa) to interact with our OpenShift cluster. Please see installation instructions below if you do not already have this deployed in your environment. 

* [OpenShift MCP installation instructions](../../../kubernetes/mcp-servers/openshift-mcp/README.md)

#### Slack MCP Server
We will also be using the [Slack MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/slack) in this notebook. Please see installation instructions below if you do not already have this deployed in your environment. 

* [Slack MCP installation instructions](../../../kubernetes/mcp-servers/slack-mcp/README.md)

### Pre-Requisites

Before starting, ensure you have the following:
- A running Llama Stack server
- A running Slack MCP server. Refer to our [documentation](https://github.com/opendatahub-io/llama-stack-demos/tree/main/kubernetes/mcp-servers/slack-mcp) on how you can set this up on your OpenShift cluster
- Access to an OpeShift cluster with a deployment of the [OpenShift MCP server](../../../kubernetes/mcp-servers/openshift-mcp) (see the [deployment manifests](https://github.com/opendatahub-io/llama-stack-on-ocp/tree/main/kubernetes/mcp-servers/openshift-mcp) for assistance with this).

## Setting Up this Notebook
We will initialize our environment as described in detail in our [\"Getting Started\" notebook](./Level0_getting_started_with_Llama_Stack.ipynb). Please refer to it for additional explanations.

In [1]:
# for communication with Llama Stack
from llama_stack_client import LlamaStackClient
from llama_stack_client import Agent
from llama_stack_client.lib.agents.event_logger import EventLogger
from llama_stack_client import RAGDocument

# pretty print of the results returned from the model/agent
import sys
sys.path.append('..')  
from src.utils import step_printer
import uuid
import os

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
base_url = os.getenv("REMOTE_BASE_URL")


# Tavily search API key is required for some of our demos and must be provided to the client upon initialization.
# We will cover it in the agentic demos that use the respective tool. Please ignore this parameter for all other demos.
tavily_search_api_key = os.getenv("TAVILY_SEARCH_API_KEY")
if tavily_search_api_key is None:
    provider_data = None
else:
    provider_data = {"tavily_search_api_key": tavily_search_api_key}


client = LlamaStackClient(
    base_url=base_url,
    provider_data=provider_data
)

print(f"Connected to Llama Stack server")

# model_id will later be used to pass the name of the desired inference model to Llama Stack Agents/Inference APIs
model_id = "granite32-8b"

temperature = float(os.getenv("TEMPERATURE", 0.0))
if temperature > 0.0:
    top_p = float(os.getenv("TOP_P", 0.95))
    strategy = {"type": "top_p", "temperature": temperature, "top_p": top_p}
else:
    strategy = {"type": "greedy"}

max_tokens = int(os.getenv("MAX_TOKENS", 512))

# sampling_params will later be used to pass the parameters to Llama Stack Agents/Inference APIs
sampling_params = {
    "strategy": strategy,
    "max_tokens": max_tokens,
}

# For this demo, we are using Milvus Lite, which is our preferred solution. Any other Vector DB supported by Llama Stack can be used.

# RAG vector DB settings
VECTOR_DB_EMBEDDING_MODEL = os.getenv("VDB_EMBEDDING")
VECTOR_DB_EMBEDDING_DIMENSION = int(os.getenv("VDB_EMBEDDING_DIMENSION", 384))
VECTOR_DB_CHUNK_SIZE = int(os.getenv("VECTOR_DB_CHUNK_SIZE", 512))
VECTOR_DB_PROVIDER_ID = os.getenv("VDB_PROVIDER")

# Unique DB ID for session
vector_db_id = f"test_vector_db_{uuid.uuid4()}"

stream_env = os.getenv("STREAM", "False")
# the Boolean 'stream' parameter will later be passed to Llama Stack Agents/Inference APIs
# any value non equal to 'False' will be considered as 'True'
stream = (stream_env != "False")

print(f"Inference Parameters:\n\tModel: {model_id}\n\tSampling Parameters: {sampling_params}\n\tstream: {stream}")

Connected to Llama Stack server
Inference Parameters:
	Model: granite32-8b
	Sampling Parameters: {'strategy': {'type': 'greedy'}, 'max_tokens': 512}
	stream: False


### Indexing the Documents
- Initialize a new document collection in the target vector DB. All parameters related to the vector DB, such as the embedding model and dimension, must be specified here.
- Provide a list of document URLs to the RAG tool. Llama Stack will handle fetching, conversion and chunking of the documents' content.

In [3]:
# define and register the document collection to be used
client.vector_dbs.register(
    vector_db_id=vector_db_id,
    embedding_model=VECTOR_DB_EMBEDDING_MODEL,
    embedding_dimension=VECTOR_DB_EMBEDDING_DIMENSION,
    provider_id=VECTOR_DB_PROVIDER_ID,
)

# ingest the documents into the newly created document collection
urls = [
    ("https://docs.redhat.com/en/documentation/openshift_container_platform/4.11/html/support/troubleshooting#troubleshooting-installations.html", "application/html"),
]
documents = [
    RAGDocument(
        document_id=f"num-{i}",
        content=url,
        mime_type=url_type,
        metadata={},
    )
    for i, (url, url_type) in enumerate(urls)
]
client.tool_runtime.rag_tool.insert(
    documents=documents,
    vector_db_id=vector_db_id,
    chunk_size_in_tokens=VECTOR_DB_CHUNK_SIZE,
)

### Validate tools are available in our llama stack instance

When an instance of llama stack is redeployed your tools need to re-registered. Also if a tool is already registered with a llama stack instance, if you try to register one with the same `toolgroup_id`, llama stack will throw you an error.

For this reason it is recommended to include some code to validate your tools and toolgroups. This is where the `mcp_url` comes into play. The following code will check that the `builtin::rag`,`mcp::openshift`  and `mcp::slack` tools are registered as tools, but if any mcp tool is not listed there, it will attempt to register it using the mcp url.

If you are running the MCP server from source, the default value for this is: `http://localhost:8000/sse`.

If you are running the MCP server from a container, the default value for this is: `http://host.containers.internal:8000/sse`.

Make sure to pass the corresponding MCP URL for the server you are trying to register/validate tools for.

In [4]:
# Optional: Enter your MCP server URL here
ocp_mcp_url = os.getenv("REMOTE_OCP_MCP_URL") # Optional: enter your MCP server url here
slack_mcp_url = os.getenv("REMOTE_SLACK_MCP_URL") # Optional: enter your MCP server url here

# Get list of registered tools and extract their toolgroup IDs
registered_tools = client.tools.list()
registered_toolgroups = [tool.toolgroup_id for tool in registered_tools]

if  "builtin::rag" not in registered_toolgroups: # Required
    client.toolgroups.register(
        toolgroup_id="builtin::rag",
        provider_id="milvus"
    )

if "mcp::openshift" not in registered_toolgroups: # required
    client.toolgroups.register(
        toolgroup_id="mcp::openshift",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":ocp_mcp_url},
    )

if "mcp::slack" not in registered_toolgroups: # required
    client.toolgroups.register(
        toolgroup_id="mcp::slack",
        provider_id="model-context-protocol",
        mcp_endpoint={"uri":slack_mcp_url},
    )

# Log the current toolgroups registered
print(f"Your Llama Stack server is already registered with the following tool groups: {set(registered_toolgroups)}\n")

Your Llama Stack server is already registered with the following tool groups: {'builtin::rag', 'builtin::websearch', 'mcp::slack', 'mcp::openshift'}



### System Prompts for different models

**Note:** If you have multiple models configured with your Llama Stack server, you can choose which one to run your queries against. When switching to a different model, you may need to adjust the system prompt to align with that model’s expected behavior. Many models provide recommended system prompts for optimal and reliable outputs these are typically documented on their respective websites.

In [5]:
model_prompt= """You are a helpful assistant. You have access to a number of tools.
Whenever a tool is called, be sure return the Response in a friendly and helpful tone."""

# Resolve and report any errors that might be happening on my OpenShift Cluster

We will also be using the [OpenShift MCP Server](https://github.com/manusa/kubernetes-mcp-server) and the [Slack MCP Server](https://github.com/modelcontextprotocol/servers/tree/main/src/slack) for this query. If you haven't already, you can follow the instructions [here](https://github.com/opendatahub-io/llama-stack-demos/tree/main/kubernetes/mcp-servers/slack-mcp#setting-up-on-ocp) to install the Slack MCP server and the instructions [here](https://github.com/opendatahub-io/llama-stack-demos/tree/main/kubernetes/mcp-servers/openshift-mcp#steps-for-deploying-the-openshift-mcp-server-on-openshift) to install the OpenShift MCP server. 

#### Additional Steps for Slack MCP Server
Once you have the Slack MCP server running, you will need to:

- Setup a slack app with your Slack workspace by following the steps [here](https://github.com/opendatahub-io/llama-stack-demos/tree/main/kubernetes/mcp-servers/slack-mcp#setting-up-the-slack-bot)
- Once your Slack app is set up and you've got the OAuth token, you will need to provide the token into your Slack MCP server to let the app post messages to your channels.
- Finally, you will need to register your Slack MCP server with your Llama Stack server

In [6]:
# Create simple agent with tools
agent = Agent(
    client,
    model=model_id, # replace this with your choice of model
    instructions = model_prompt , # update system prompt based on the model you are using
    tools=[dict(
            name="builtin::rag",
            args={
                "vector_db_ids": [vector_db_id],  # list of IDs of document collections to consider during retrieval
            },
        ),"mcp::openshift", "mcp::slack"],
    tool_config={"tool_choice":"auto"},
    sampling_params=sampling_params
)

user_prompts = ["View the logs for pod slack-test in the llama-serve OpenShift namespace. Categorize it as normal or error.",
                "Search for solutions on this error and provide a summary of the steps to take in just 1-2 sentences.",
               "Send a message with the summarization to the demos channel on Slack."]
session_id = agent.create_session(session_name="OCP_Slack_demo")
for i, prompt in enumerate(user_prompts):
    response = agent.create_turn(
        messages=[
            {
                "role":"user",
                "content": prompt
            }
        ],
        session_id=session_id,
        stream=stream,
    )
    if stream:
        for log in EventLogger().log(response):
            log.print()
    else:
        step_printer(response.steps) # print the steps of an agent's response in a formatted way. 


---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: pods_log, Arguments: {'name': 'slack-test', 'namespace': 'llama-serve'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


{
│   'type': 'text',
│   'text': 'Starting container...\nFailure: Unknown Error\nError details: Container failed due to an unexpected issue during startup.\nPotential cause: Missing dependencies, configuration errors, or permission issues.\n',
│   'annotations': None
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The logs for the pod `slack-test` in the `llama-serve` OpenShift namespace indicate an error. The container failed due to an unexpected issue during startup. This could be due to missing dependencies, configuration errors, or permission issues.

========== Query processing completed ========== 


---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: knowledge_search, Arguments: {'query': 'Container failed due to an unexpected issue during startup'}

---------- 📍 Step 2: ToolExecutionStep ----------
🔧 Executing tool...


[
│   TextContentItem(
│   │   text='knowledge_search tool found 5 chunks:\nBEGIN of knowledge_search tool results.\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 1:\nDocument_id:num-0\nContent: RunInitContainer</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tContainer initialization failed.\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td></tr><tr><td align="left" valign="top" headers="idm140345532084640"> <p>\n\t\t\t\t\t\t\t\t\t<code class="literal">ErrRunContainer</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tNone of the pod’s containers started successfully.\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td></tr><tr><td align="left" valign="top" headers="idm140345532084640"> <p>\n\t\t\t\t\t\t\t\t\t<code class="literal">ErrKillContainer</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tNone of the pod’s containers were killed successfully.\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td></tr><tr><td align="left" valign="top" headers="idm140345532084640"> <p>\n\t\t\t\t\t\t\t\t\t<code class="literal">ErrCrashLoopBackOff</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tA container has terminated. The kubelet will not attempt to restart it.\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td></tr><tr><td align="left" valign="top" headers="idm140345532084640"> <p>\n\t\t\t\t\t\t\t\t\t<code class="literal">ErrVerifyNonRoot</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tA container or image attempted to run with root privileges.\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td></tr><tr><td align="left" valign="top" headers="idm140345532084640"> <p>\n\t\t\t\t\t\t\t\t\t<code class="literal">ErrCreatePodSandbox</code>\n\t\t\t\t\t\t\t\t</p>\n\t\t\t\t\t\t\t\t </td><td align="left" valign="top" headers="idm140345532083552"> <p>\n\t\t\t\t\t\t\t\t\tPod sandbox creation did not succeed.\n\t\t\t\t\t\t\t\t</p>\n\n',
│   │   type='text'
│   ),
│   TextContentItem(
│   │   text='Result 2:\nDocument_id:num-0\nContent: \\tYou cannot create a new container on a working node and the “can’t stat lower layer” error appears:\\n\\t\\t\\t\\t\\t\\t\\u003C/p>\\u003Cpre class=\\"programlisting language-terminal\\">can\'t stat lower layer ...  because it does not exist.  Going through storage to recreate the missing symlinks.\\u003C/pre>\\u003C/li>\\u003Cli class=\\"listitem\\">\\n\\t\\t\\t\\t\\t\\t\\tYour node is in the \\u003Ccode class=\\"literal\\">NotReady\\u003C/code> state after a cluster upgrade or if you attempt to reboot it.\\n\\t\\t\\t\\t\\t\\t\\u003C/li>\\u003Cli class=\\"listitem\\">\\n\\t\\t\\t\\t\\t\\t\\tThe container runtime implementation (\\u003Ccode class=\\"literal\\">crio\\u003C/code>) is not working properly.\\n\\t\\t\\t\\t\\t\\t\\u003C/li>\\u003Cli class=\\"listitem\\">\\n\\t\\t\\t\\t\\t\\t\\tYou are unable to start a debug shell on the node using \\u003Ccode class=\\"literal\\">oc debug node/&lt;node_name&gt;\\u003C/code> because the container runtime instance (\\u003Ccode class=\\"literal\\">crio\\u003C/code>) is not working.\\n\\t\\t\\t\\t\\t\\t\\u003C/li>\\u003C/ul>\\u003C/div>\\u003Cp>\\n\\t\\t\\t\\t\\tFollow this process to completely wipe the CRI-O storage and resolve the errors.\\n\\t\\t\\t\\t\\u003C/p>\\u003Cdiv class=\\"itemizedlist\\">\\u003Cp class=\\"title\\">\\u003Cstrong>Prerequisites:\\u003C/strong>\\u003C/p>\\u003Cul class=\\"itemizedlist\\" type=\\"disc\\">\\u003Cli class=\\"listitem\\">\\n\\t\\t\\t\\t\\t\\t\\tYou have access to the cluster as a user with the \\u003Ccode class=\\"literal\\">cluster-admin\\u003C/code> role.\\n\\t\\t\\t\\t\\t\\t\\u003C/li>\\u003Cli class=\\"listitem\\">\\n


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
Based on the search results, here's a summary:

To resolve the "Container failed due to an unexpected issue during startup" error, consider the following steps:

1. Check if the issue is related to missing dependencies, configuration errors, or permission issues.
2. Verify if the container runtime (like CRI-O) is functioning correctly.
3. If the node is in "NotReady" state after a cluster upgrade or reboot, troubleshoot accordingly.
4. If the error persists, consider wiping the CRI-O storage completely, following the provided process.

Remember, these steps are general suggestions based on the search results. The exact solution may vary depending on your specific setup and environment.

========== Query processing completed ========== 


---------- 📍 Step 1: InferenceStep ----------
🛠️ Tool call Generated:
Tool call: slack_post_message, Arguments: {'channel_id': 'demos', 'text': "Container failed due to an unexpected issu

{
│   'type': 'text',
│   'text': '{"ok":true,"channel":"C08MUDSNHED","ts":"1747095555.646859","message":{"user":"U08MM2P7BEF","type":"message","ts":"1747095555.646859","bot_id":"B08MM2P60KV","app_id":"A08MLQCUGUS","text":"Container failed due to an unexpected issue during startup. Here\'s a summary of potential steps to resolve: 1) Check for missing dependencies, configuration errors, or permission issues. 2) Verify the functioning of the container runtime (like CRI-O). 3) Troubleshoot if the node is in \'NotReady\' state after a cluster upgrade or reboot. 4) If needed, consider wiping the CRI-O storage completely. Note: These are general suggestions, and the exact solution may vary based on your specific setup and environment.","team":"T08M9UTL2DC","bot_profile":{"id":"B08MM2P60KV","app_id":"A08MLQCUGUS","user_id":"U08MM2P7BEF","name":"et-slack-bot","icons":{"image_36":"https://a.slack-edge.com/80588/img/plugins/app/bot_36.png","image_48":"https://a.slack-edge.com/80588/img/plugins/app/bot_48.png","image_72":"https://a.slack-edge.com/80588/img/plugins/app/service_72.png"},"deleted":false,"updated":1744227670,"team_id":"T08M9UTL2DC"},"blocks":[{"type":"rich_text","block_id":"Bri57","elements":[{"type":"rich_text_section","elements":[{"type":"text","text":"Container failed due to an unexpected issue during startup. Here\'s a summary of potential steps to resolve: 1) Check for missing dependencies, configuration errors, or permission issues. 2) Verify the functioning of the container runtime (like CRI-O). 3) Troubleshoot if the node is in \'NotReady\' state after a cluster upgrade or reboot. 4) If needed, consider wiping the CRI-O storage completely. Note: These are general suggestions, and the exact solution may vary based on your specific setup and environment."}]}]}]},"warning":"missing_charset","response_metadata":{"warnings":["missing_charset"]}}',
│   'annotations': None
}


---------- 📍 Step 3: InferenceStep ----------
🤖 Model Response:
The message has been successfully posted to the "demos" channel on Slack. Here's the message:

"Container failed due to an unexpected issue during startup. Here's a summary of potential steps to resolve: 1) Check for missing dependencies, configuration errors, or permission issues. 2) Verify the functioning of the container runtime (like CRI-O). 3) Troubleshoot if the node is in 'NotReady' state after a cluster upgrade or reboot. 4) If needed, consider wiping the CRI-O storage completely. Note: These are general suggestions, and the exact solution may vary based on your specific setup and environment."

========== Query processing completed ========== 



### Output Analysis

Lets step through the output to further understands whats happening in this Agentic demo.

1. First the LLM sends off a tool call to the `pods_log` tool configured with the OpenShift MCP server, to fetch the logs for the pod specified from the OpenShift cluster.
2. The tool successfully retrieves the logs for the pod.
3. The LLM recieves the response from the tool call, which are the pod logs, along with the original query and using its own knowledge tries to categorize it as 'Normal or 'Error'.
4. Next, the LLM sends a tool call to the RAG tool, to query the vector DB on the documents for the error from the pod logs and look for a solution to resolve the error from the document.
5. The LLM recieves the response from the tool call, and summarizes the information it retreived from the document.
6. Finally, the LLM sends the summarization as a message on Slack using the `slack_post_message` tool call configured with the Slack MCP server.

## Key Takeaways

This tutorial demonstrates how to implement agentic RAG and MCP applications with Llama Stack. We do so by initializing an agent while giving it access to the MCP tools, and RAG tool configured with Llama Stack, then invoking the agent on the specified query. Please check out our other notebooks for more examples using Llama Stack.